In [1]:
import dill
import functools
import pprint as pp
import shlex
import os
import inspect
from subprocess import run

In [102]:
def memory_profile(f):
    def unwrap_source_code(sourcelines):
        for idx, i in enumerate(sourcelines[0]):
            if i.startswith('def'):
                return "".join(sourcelines[0][idx:])
    @functools.wraps(f)
    def inner(*args, **kwargs):
        temp = './temp_ram.py'
        with open(temp, 'w+') as g:
            decorator = '@profile'
            sourcelines = inspect.getsourcelines(f)
            value = f"""import argparse\n\n{decorator}\n{unwrap_source_code(sourcelines)}"""
            g.write(value)
            g.write('\n')
            # TODO: fix the arguments
            # for parsing in case we have *args and **kwargs
            main_exec = f"""if __name__ == "__main__":     parser = argparse.ArgumentParser();    parser.add_argument('--N', required=True);    args = parser.parse_args();    {f.__name__}(args.N)
                    """
            g.write(main_exec)
            
        cmd = shlex.split(f'python -m memory_profiler {temp} --N 10')
        mem_stats = run(cmd, capture_output=True).stdout.decode('utf-8')
        os.remove(temp)
        print(mem_stats)
        return f(*args, **kwargs)
    return inner
    
    
    
def timing(f):
    import time
    @functools.wraps(f)
    def inner(*args, **kwargs):
        start = time.perf_counter()
        value = f(*args, **kwargs)
        end = time.perf_counter()
        print(f"Finished {f.__name__} in {end - start:.4f} seconds.")
        return value
    return inner

In [103]:
@memory_profile
@timing
def QWERTY(x):
    res = []
    for i in range(x):
        res.append(i)
    return res


print(QWERTY(10))

Filename: ./temp_ram.py

Line #    Mem usage    Increment   Line Contents
     3   40.195 MiB   40.195 MiB   @profile
     4                             def QWERTY(x):
     5   40.195 MiB    0.000 MiB       res = []
     6   40.199 MiB    0.004 MiB       for i in range(x):
     7                                     res.append(i)
     8                                 return res



Finished QWERTY in 0.0000 seconds.
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
